<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')
target = pd.read_csv('/content/target.csv', encoding = 'cp949')

In [ ]:
!pip install autokeras

In [4]:
import tensorflow as tf
import autokeras as ak

Using TensorFlow backend


In [5]:
# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

current_year = datetime.now().year

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

In [6]:
# 계절 분류 함수 정의
def classify_season(month):
    if  month == 1 or month == 2 or month == 12:
        return '겨울'
    elif 3 <= month <= 5:
        return '봄'
    elif 6 <= month <= 8:
        return '여름'
    else:
        return '가을'

# '일시' 컬럼에서 월 정보를 추출하여 '계절' 열 추가
data['월'] = data['일시'].dt.month
data['계절'] = data['월'].apply(classify_season)

# 계절별 중앙값 계산
seasonal_medians = data.groupby('계절').transform('median')

# 결측값 대체
data.fillna(seasonal_medians, inplace=True)

In [7]:
# 입력 특성과 출력 정의
x_train = data.drop(columns=['평균기온'])  # 입력 특성
y_train = data['평균기온']  # 출력 정답

# AutoML 모델 정의
regressor = ak.StructuredDataRegressor(max_trials=1)  # max_trials는 조정 가능

# 모델 훈련
regressor.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 00m 45s]
val_loss: 27.817323684692383

Best val_loss So Far: 27.817323684692383
Total elapsed time: 00h 00m 45s
Epoch 1/10
719/719 [==============================] - 4s 4ms/step - loss: 71.3991 - mean_squared_error: 71.3991
Epoch 2/10
719/719 [==============================] - 2s 2ms/step - loss: 26.5444 - mean_squared_error: 26.5444
Epoch 3/10
719/719 [==============================] - 2s 2ms/step - loss: 22.8874 - mean_squared_error: 22.8874
Epoch 4/10
719/719 [==============================] - 2s 2ms/step - loss: 20.2050 - mean_squared_error: 20.2050
Epoch 5/10
719/719 [==============================] - 2s 3ms/step - loss: 17.9133 - mean_squared_error: 17.9133
Epoch 6/10
719/719 [==============================] - 3s 4ms/step - loss: 15.9846 - mean_squared_error: 15.9846
Epoch 7/10
719/719 [==============================] - 2s 2ms/step - loss: 14.4187 - mean_squared_error: 14.4187
Epoch 8/10
719/719 [==============================] - 2s 2ms/step - loss: 13.1158 

In [8]:
# AutoKeras 모델 확인
model = regressor.export_model()  # AutoKeras 모델을 내보내서 TensorFlow 모델로 변환
model.summary()  # TensorFlow 모델의 구조 확인

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12)]              0         
                                                                 
 multi_category_encoding (M  (None, 12)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 12)                25        
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                416       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

In [9]:
reg_pred = regressor.predict(x_train)

719/719 [==============================] - 1s 2ms/step


In [10]:
import numpy as np

# 주어진 배열
input_array = np.array(reg_pred)  # 22995개의 행

# 365행을 갖는 62개의 열로 변환
output_array = input_array.reshape((63, 365))

In [12]:
# 각 열별로 중앙값 계산하여 365개의 행으로 만들기
medians = np.median(output_array, axis=0)
result_array = medians.reshape((365, 1))

In [13]:
submission['평균기온'] = result_array[:358]
print(submission.head(30))

            일시      평균기온
0   2023-01-01 -0.862728
1   2023-01-02 -0.564053
2   2023-01-03 -1.318483
3   2023-01-04 -2.009847
4   2023-01-05 -1.793496
5   2023-01-06 -1.167435
6   2023-01-07 -1.280011
7   2023-01-08 -1.608852
8   2023-01-09 -1.439035
9   2023-01-10 -0.779074
10  2023-01-11 -1.897509
11  2023-01-12 -2.256212
12  2023-01-13 -2.725864
13  2023-01-14 -3.362925
14  2023-01-15 -1.616807
15  2023-01-16 -1.159448
16  2023-01-17 -2.077671
17  2023-01-18 -2.454230
18  2023-01-19 -3.174580
19  2023-01-20 -1.644116
20  2023-01-21 -1.484362
21  2023-01-22 -1.483323
22  2023-01-23 -1.690084
23  2023-01-24 -2.171626
24  2023-01-25 -1.183056
25  2023-01-26 -1.785281
26  2023-01-27 -1.322183
27  2023-01-28 -1.238248
28  2023-01-29 -0.873744
29  2023-01-30 -1.391449


In [14]:
submission.to_csv("DEAGYUL_26_submission.csv", index=False)

print(submission.head())

           일시      평균기온
0  2023-01-01 -0.862728
1  2023-01-02 -0.564053
2  2023-01-03 -1.318483
3  2023-01-04 -2.009847
4  2023-01-05 -1.793496
